# Word Embeddings

In Notebook 1, we represent a document with a vector whose size is equal to the the size of vocabulary set. We encoded each words with one hot encoding technique which assign a value in the vector at index corresponding to the vocabulary and leave other elements zero. This technique has several drawbacks. It creates sparseness in vector space. It also cannot capture two different words that are synonym or similar, or share some sort of relations. For example, word `cat` and `dog` will totally different, as different as words like `electrical` and `poem`. These weaknesses can undermine downstream tasks. To solve these issues, researcher comes up with [dense representation](https://web.stanford.edu/~jurafsky/slp3/6.pdf) to be contrast with the sparseness of one hot encoding. Several approaches on dense representation have been studies since 1990s and culminated at the invention of [Word2Vec](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) in 2013. The technique (seemingly, since it's still controversial) outperforms previous dense representation techniques discovered in 1990s for many downstream tasks. Our experiment will be center on pre-trained [Word2Vec] by exploring different ways of using it and their performances.

    
**Prerequisite**

1. Download [Google Word2Vec Model](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing) to this directory and run 

    ```
    gunzip GoogleNews-vectors-negative300.bin.gz
    ```

2. Download [Stanford GloVe Model](http://nlp.stanford.edu/data/glove.840B.300d.zip) to this directory and run the following commands.

    ```
    unzip glove.840B.300d.zip
    python -m gensim.scripts.glove2word2vec --input glove.840B.300d.txt --output glove.840B.300d.w2vformat.txt
    ```

GloVe is also available in SpaCy's `en_core_web_md` too. See [Document](https://spacy.io/models/en#en_core_web_md). In this notebook, we will not use GloVe from SpaCy due to lots of its limitations.

If you already have those files or you don't want to save it in this directory, you can either change constant variable PRETRAINED_WV_MODEL_PATH  and PRETRAINED_GLOVE_MODEL_PATH or create symbolic link.
    
```
ln -s /path/to/your/word2vec ./GoogleNews-vectors-negative300.bin
ln -s /path/to/your/glove ./glove.840B.300d.w2vformat.txt

```

In [1]:
%load_ext autoreload
%autoreload

from lib.dataset import download_tfds_imdb_as_text, download_tfds_imdb_as_text_tiny
from lib.word_emb import run_pipeline
import gensim

PRETRAINED_WV_MODEL_PATH = "./GoogleNews-vectors-negative300.bin"
PRETRAINED_GLOVE_MODEL_PATH = "./glove.840B.300d.w2vformat.txt"


In [2]:
# load pre-trained word embeddings from disk - take about 10 mins to run

word_emb_models = {
    "word2vec": gensim.models.KeyedVectors.load_word2vec_format(PRETRAINED_WV_MODEL_PATH, binary=True),
    "glove": gensim.models.KeyedVectors.load_word2vec_format(PRETRAINED_GLOVE_MODEL_PATH, binary=False) 
}

# load dataset

dataset  = download_tfds_imdb_as_text()
tiny_dataset = download_tfds_imdb_as_text_tiny() 


# Experiment 1 - Text preprocessing

In Notebook 1, we do text preprocessing to reduce number of features. For example, words like `Movie`, `movie` and `movies` share the same meaning and play the same role in classification model, so we do lowercase and lemmatization to group them together. We also remove stop words and number because we believe that they don't play any roles in classification mode. However, this is not necessary the case for word embeddings. In pre-trained Word2Vec, it has embeddings for `Movie`, `movie` and `movies` and those embeddings are very similar. Furthermore, embeddings can encode subtle different of words like `good`, `better` and `best`. We will see if text processing is not necessary for text classification with word embeddings. 

In [3]:
# take about 20 mins to run
    
print("Simple SpaCy tokenizer")
_, _ = run_pipeline(dataset, word_emb_models["word2vec"])

print("Simple SpaCy tokenizer and lowercase")
_, _ = run_pipeline(dataset, word_emb_models["word2vec"], lower=True)

print("Simple SpaCy tokenizer, lowercase, ignore stop words and numbers")
_, _ = run_pipeline(dataset, word_emb_models["word2vec"], lower=True, ignore=["like_num", "is_stop"])


Simple SpaCy tokenizer
Best parameters set found on development set:  {'C': 10}
Best F1 on development set: 0.85
F1 on test set: 0.85
time: 566.28
Simple SpaCy tokenizer and lowercase
Best parameters set found on development set:  {'C': 100}
Best F1 on development set: 0.85
F1 on test set: 0.85
time: 319.10
Simple SpaCy tokenizer, lowercase, ignore stop words and numbers
Best parameters set found on development set:  {'C': 1000}
Best F1 on development set: 0.85
F1 on test set: 0.85
time: 282.63


Here we see that we get F1 around 0.85 regardless of preprocessing. Ignoring stop words even worsen the F1. The result suggest that preprocessing is not crucial if we use word embeddings. Actually, most NLP problems with very large dataset that are solved with Deep Learning that I've seen so far do not focus on preprocessing like removing stop words, lemmatization that much. We will use the result from "Simple SpaCy tokenizer" as baseline for later experiments in the Notebook.

# Experiment 2 - Embeddings

In this experiment, we will use two different word embeddings, [Word2Vec](https://arxiv.org/pdf/1310.4546.pdf) and [GloVE](https://nlp.stanford.edu/projects/glove/). The high level intuitions of both embeddings are similar in the sense that they both estimate dense representation of words based on co-occurrence, i.e. words that are replaceable are similar. However, their models are very different. In a nutshell, GloVE directly estimates embeddings from co-occurrence matrix, while Word2Vec is a learning based model that learns to predict neighboring words from center words (skip-gram) or other way around (C-BOW). More info, see [this](https://www.quora.com/How-is-GloVe-different-from-word2vec).

We will use pre-trained Word2Vec and GloVE. The pre-trained Word2Vec has 3M words, trained on roughly 100B tokens from a Google News dataset. The vector length is 300 features. More info, see [this](https://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/). The pre-trained GloVE model has 2.2M words, trained on 840B tokens from Common Crawl. The vector length is also 300 features. In sum
- both trained on very large corpus (100B vs 840B)
- both trained on general corpus (Google News vs Common Crawl)
- both has 300 features


Also note that differences of embeddings in this experiment is not only the models (Word2Vec vs GloVE) but also the data they were trained. 



In [4]:
# take about 6 mins to run 

print("GloVe")
_, _ = run_pipeline(dataset, word_emb_models["glove"])
    

GloVe
Best parameters set found on development set:  {'C': 1000}
Best F1 on development set: 0.85
F1 on test set: 0.84
time: 343.38


While both model are trained on pretty similar data (in term of amount and type), GloVe performs slightly worse than the baseline (0.84 to 0.85). 

# Experiment 3 - TFIDF

[TFIDF](https://nlp.stanford.edu/IR-book/pdf/06vect.pdf) is another well-known technique for Information Retrieval. The idea is that the words that present frequently across document e.g. `is`, `that` convey less information that words that rarely present, so they should be assigned less number. However, since text classification is supervise learning, it is possible that the classification model can learn the same information (which words, or which feature in vector space, are more important than others). In this experiment, we will see if TFIDF is useful for classification problem.

In [5]:
# take about 6 mins to run

print("idf")
_, _ = run_pipeline(dataset, word_emb_models["word2vec"], tfidf=True)


idf
Best parameters set found on development set:  {'C': 1000}
Best F1 on development set: 0.84
F1 on test set: 0.83
time: 346.09


TFIDF has been long proved that works well for Information Retrieval task. However, it is not necessary for text classification since we observe the lower F1 than the baseline. It's better to let model learn which features are important than add this piece of information on the features.

Another way to think of this is that our dataset (IMDB review) is not long enough as we can see in Notebook 0. They also do not usually present as many repetitive words so that it can take advantage of log pooling.

# Experiment 4 - Pooling

While reading the first three experiments in this Notebook, you may be wondering that Word embeddings (Word2Vec and GloVE) are dense representation of "words" not "document", so how can we come up with vectors representing document? To do so, we have to pool word embeddings, similar idea as pooling layer in CNN. In the first three experiments, we simply average the embeddings of each tokens to get the vector representing the document. While this technique is so simple, it has been widely used. Not only in academic, but industrial NLP library such as spaCy [doc vector](https://spacy.io/api/doc#vector) and [BERT-AS-A-SERVICE](https://github.com/hanxiao/bert-as-service#speech_balloon-faq) also pool a document vector by averaging. 


However, averaging is not the only way we can pool a document vector. Let's step back a little to the fundamental. What do we do in Notebook 1? We use one-hot encoding to encode a word and then we sum them up! Although our word representation is now embeddings (dense) instead of one-hot encoding (sparse), we can still do the same thing. The reason why averaging is more popular is that it eliminate the effect of document length. For example, these two documents `cat cat dog dog` and `cat dog` will be the same in vector space. Another technique is to use log, as presented in this [book](https://nlp.stanford.edu/IR-book/pdf/06vect.pdf). The idea is to reduce the effect of token that occur many times. For example, the document like `dog dog dog cat` will lean toward `dog` in vector space if we average.  However, it will lean toward `dog` in less degree if we use log pooling technique. However, this log technique is introduced for Information Retrieval context, which is determining the query vector and document vector. As our problem set is text classification, this technique may not work. 

One may speculate that averaging and summing are pretty much the same since we just multiply vectors with some constants. This may be true for information retrieval since `sim(q, d)` and `sim(q, c x d)` are the same where `sim` is cosine similarity. However, for classification we are to draw a boundary in vector space, and since by taking average we multiply those vectors with different constants (each document can have different length), it can change to decision boundary. 

Note that all these variations are Bag Of Word, which does not take the position of words into account. In other words `The movie is not good. It is boring` and `The movie is not boring. It is good` are represented with the same vector.

In this experiment, we will try three pooling technique: sum, average and log.



    

In [6]:
# take about 55 mins to run

print("sum")
_, _ = run_pipeline(dataset, word_emb_models["word2vec"], polling="sum")

print("log")
_, _ = run_pipeline(dataset, word_emb_models["word2vec"], polling="log")
    

sum
Best parameters set found on development set:  {'C': 0.001}
Best F1 on development set: 0.85
F1 on test set: 0.85
time: 2018.71
log
Best parameters set found on development set:  {'C': 0.001}
Best F1 on development set: 0.85
F1 on test set: 0.85
time: 1215.01


Here it turns out that we don't observe any different from different pooling technique.

In these four experiments in this Notebook, we see F1 score is significantly lower than Logistic Regression Model in Notebook 1. Let's discuss possible reasons. There is one important point that has (intentionally) not been discussed - what to do when tokens in document not present in pre-trained Word2Vec (or GloVE)? There are several options
- ignore (assign zero) as if these tokens do not present in the document. Library like spaCy [doc vector](https://spacy.io/api/doc#vector) and [BERT-AS-A-SERVICE](https://github.com/hanxiao/bert-as-service#speech_balloon-faq) choose this approach
- assigned small randomized vector, this is to prevent zero vector if all tokens in the document do not present in pre-trained word embeddings
- some pre-trained word embeddings designate special token let's say `UNK` to represent rare tokens in corpus it was trained on. For example, when training, it can substitute tokens whose occurrence less than 5 with `UNK` because they are two rare to be trained and deliver meaningful embeddings. It's hard to say what linguistic property encoded in `UNK` embedding. If it encode "rareness", then it makes sense that we use this embedding with we see token not present in the pre-trained word embeddings vocabulary set. The pre-trained Word2Vec actually has token called `UNK` but unfortunately I can't find any document to confirm that this token is special token designated to rare words. Therefore, I do not use this approach.
- randomly pick an embedding from the pre-trained word embeddings. This is what I use in all experiments in this Notebook.

Let's inspect the vectorizer and see how many words in dataset that do not present in pre-trained Word2Vec.



In [9]:
_, dense_vectorizer = run_pipeline(dataset, word_emb_models["word2vec"])
dense_vectorizer.print_stat()

Best parameters set found on development set:  {'C': 100}
Best F1 on development set: 0.85
F1 on test set: 0.85
time: 317.04
fit_transform
oov freq 0.21236471197520704
%unk in vocab 0.005534666666666667
transform
oov freq 0.21268567382982578
%unk in vocab 0.005533666666666667


Now we know what's the problem. We only use about 80% of information from the dataset in classification model. We discard 20% of information completely. In the Notebook 3, we will study how can we use the information more efficiently.